In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def row_to_df(row):
    t = row.iloc[0]
    time_index = pd.date_range(start=t+pd.Timedelta(minutes=5),end=t+pd.Timedelta(days=3),freq='5min')
    df = pd.DataFrame({'Datetime': time_index, 'Ihat': row.iloc[1:]})
    return df

def cal_metrics_daytime(bot_df,col_meas,day):
    mbe = 0; mae = 0; rmse = 0;smape = 0
    n = 0
    day = [i+1 for i in day]

    for i in bot_df.columns[day]:
        merge_df = bot_df.loc[:, ['Datetime', col_meas, i]]
        merge_df.dropna(inplace=True)
        merge_df.set_index('Datetime',inplace=True)
        merge_df = merge_df.between_time('06:00','18:00')

        mbe += (merge_df[i] - merge_df[col_meas]).sum()
        mae += (abs(merge_df[i] - merge_df[col_meas])).sum()
        rmse += ((merge_df[i] - merge_df[col_meas]) ** 2).sum()
        n += merge_df[col_meas].count()

        merge_df = merge_df[~((abs(merge_df[col_meas]) <= 10**-4) & (abs(merge_df[i] <= 10**-4)))]
        smape += (abs(merge_df[i] - merge_df[col_meas]) / ((abs(merge_df[i]) + abs(merge_df[col_meas]))/2)).sum()

    mae = mae/n; rmse = np.sqrt(rmse/n); mbe = mbe/n ; smape = smape*100/n

    return [mae, rmse, mbe,smape,n]


def cal_metrics_hour_daytime(bot_df,col_meas,hour): # hour from 6:00
    mbe = 0; mae = 0; rmse = 0;smape = 0
    n = 0

    merge_df = bot_df.loc[:, ['Datetime', col_meas, 'day1']]
    merge_df.dropna(inplace=True)
    merge_df.set_index('Datetime',inplace=True)
    merge_df = merge_df.between_time('06:00',f'{6 + hour:02d}:00')

    mbe += (merge_df['day1'] - merge_df[col_meas]).sum()
    mae += (abs(merge_df['day1'] - merge_df[col_meas])).sum()
    rmse += ((merge_df['day1'] - merge_df[col_meas]) ** 2).sum()
    n += merge_df[col_meas].count()

    merge_df = merge_df[~((abs(merge_df[col_meas]) <= 10**-4) & (abs(merge_df['day1'] <= 10**-4)))]
    smape += (abs(merge_df['day1'] - merge_df[col_meas]) / ((abs(merge_df['day1']) + abs(merge_df[col_meas]))/2)).sum()

    mae = mae/n; rmse = np.sqrt(rmse/n); mbe = mbe/n ; smape = smape*100/n

    return [mae, rmse, mbe,smape,n]

In [ ]:
# timesFM solar score (daytime, day ahead)

table1 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'./timesFM/solar_3day_{i}daycontext_5min_BOT.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'Irradiance (W/m2)_imputed',[1])
    table1 = pd.concat([table1, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table1 = table1.set_index('context').T

table2 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'./timesFM/solar_3day_{i}daycontext_5min_BOT.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'Irradiance (W/m2)_imputed',[2])
    table2 = pd.concat([table2, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table2 = table2.set_index('context').T

table3 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'./timesFM/solar_3day_{i}daycontext_5min_BOT.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'Irradiance (W/m2)_imputed',[3])
    table3 = pd.concat([table3, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table3 = table3.set_index('context').T

table = pd.concat([table1,table2,table3],keys=['1 day ahead','2 day ahead','3 day ahead'],axis=0)
table

In [ ]:
# timeGPT's solar score (daytime, day ahead)

table1 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'basedontargetlag{i}dayLoad.csv',parse_dates=['Datetime'])
    # df = pd.read_csv(f'basedontargetlag{i}daySolar.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'Load(t)',[1])
    # mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'I(t)',[1])
    table1 = pd.concat([table1, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table1 = table1.set_index('context').T

table2 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'basedontargetlag{i}dayLoad.csv',parse_dates=['Datetime'])
    # df = pd.read_csv(f'basedontargetlag{i}daySolar.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'Load(t)',[2])
    # mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'I(t)',[2])
    table2 = pd.concat([table2, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table2 = table2.set_index('context').T

table3 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'basedontargetlag{i}dayLoad.csv',parse_dates=['Datetime'])
    # df = pd.read_csv(f'basedontargetlag{i}daySolar.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'Load(t)',[3])
    # mae, rmse, mbe, smape,n = cal_metrics_daytime(df,'I(t)',[3])
    table3 = pd.concat([table3, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table3 = table3.set_index('context').T

table = pd.concat([table1,table2,table3],keys=['1 day ahead','2 day ahead','3 day ahead'],axis=0)
table

context               lag 1 day     lag 3 day     lag 7 day
1 day ahead MAE        3.231313      3.174102      2.896502
            RMSE       4.613133      4.462435      3.990745
            MBE        0.521872      0.656598      0.527126
            SMAPE     61.182377     61.012849     61.217305
            n      52635.000000  52635.000000  52635.000000
2 day ahead MAE        3.960984      3.749295      3.064984
            RMSE       5.558312      5.164951      4.236384
            MBE        0.340574      0.874434      0.345280
            SMAPE     70.832697     68.128546     64.536262
            n      52635.000000  52635.000000  52635.000000
3 day ahead MAE        4.337372      4.001702      3.090582
            RMSE       5.981961      5.501663      4.304863
            MBE        0.271040      0.717643      0.275066
            SMAPE     75.388691     71.674874     63.591090
            n      52635.000000  52635.000000  52635.000000

In [ ]:
# timesFM solar score (daytime)

table1 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'./timesFM/solar_3day_{i}daycontext_5min_BOT.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'Irradiance (W/m2)_imputed',1)
    table1 = pd.concat([table1, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table1 = table1.set_index('context').T

table2 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'./timesFM/solar_3day_{i}daycontext_5min_BOT.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'Irradiance (W/m2)_imputed',4)
    table2 = pd.concat([table2, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table2 = table2.set_index('context').T

table3 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'./timesFM/solar_3day_{i}daycontext_5min_BOT.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'Irradiance (W/m2)_imputed',6)
    table3 = pd.concat([table3, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table3 = table3.set_index('context').T

table4 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'./timesFM/solar_3day_{i}daycontext_5min_BOT.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'Irradiance (W/m2)_imputed',12)
    table4 = pd.concat([table4, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)
    
table4 = table4.set_index('context').T

table = pd.concat([table1,table2,table3,table4],keys=['1 hour hori','4 hour hori','6 hour hori','12 hour hori'],axis=0)
table

In [ ]:
# timesGPT solar score (daytime)

table1 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'basedontargetlag{i}dayLoad.csv',parse_dates=['Datetime'])
    #df = pd.read_csv(f'basedontargetlag{i}daySolar.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'Load(t)',1)
    #mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'I(t)',1)
    table1 = pd.concat([table1, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table1 = table1.set_index('context').T

table2 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'basedontargetlag{i}dayLoad.csv',parse_dates=['Datetime'])
    # df = pd.read_csv(f'basedontargetlag{i}daySolor.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'Load(t)',4)
    #mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'I(t)',4)
    table2 = pd.concat([table2, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table2 = table2.set_index('context').T

table3 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'basedontargetlag{i}dayLoad.csv',parse_dates=['Datetime'])
    #df = pd.read_csv(f'basedontargetlag{i}daySolar.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'Load(t)',6)
    #mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'I(t)',6)
    table3 = pd.concat([table3, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)

table3 = table3.set_index('context').T

table4 = pd.DataFrame()
for i in [1,3,7]:
    df = pd.read_csv(f'basedontargetlag{i}dayLoad.csv',parse_dates=['Datetime'])
    #df = pd.read_csv(f'basedontargetlag{i}daySolar.csv',parse_dates=['Datetime'])
    mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'Load(t)',12)
    #mae, rmse, mbe, smape,n = cal_metrics_hour_daytime(df,'I(t)',12)
    table4 = pd.concat([table4, pd.DataFrame({'context':[f'lag {i} day'],
                                            'MAE':[mae],'RMSE':[rmse],'MBE':[mbe],'SMAPE':[smape],'n':[n]})], ignore_index=True)
    
table4 = table4.set_index('context').T

table = pd.concat([table1,table2,table3,table4],keys=['1 hour hori','4 hour hori','6 hour hori','12 hour hori'],axis=0)
table

context                lag 1 day     lag 3 day     lag 7 day
1 hour hori  MAE        1.261592      1.269468      1.408497
             RMSE       1.648496      1.665014      1.841390
             MBE        0.874898      0.850349      0.933665
             SMAPE     75.080836     77.987438     87.774890
             n       4719.000000   4719.000000   4719.000000
4 hour hori  MAE        2.190741      2.186430      2.194784
             RMSE       3.058266      3.035777      2.991244
             MBE        1.550092      1.544405      1.514104
             SMAPE     77.337135     78.806028     84.172600
             n      17787.000000  17787.000000  17787.000000
6 hour hori  MAE        2.524526      2.526891      2.453365
             RMSE       3.493749      3.494594      3.301085
             MBE        1.381771      1.434347      1.323282
             SMAPE     70.831816     71.715769     75.177339
             n      26499.000000  26499.000000  26499.000000
12 hour hori MAE        3.231313      3.174102      2.896502
             RMSE       4.613133      4.462435      3.990745
             MBE        0.521872      0.656598      0.527126
             SMAPE     61.182377     61.012849     61.217305
             n      52635.000000  52635.000000  52635.000000